# Additional End of week Exercise - week 2

Now use everything you've learned from Week 2 to build a full prototype for the technical question/answerer you built in Week 1 Exercise.

This should include a Gradio UI, streaming, use of the system prompt to add expertise, and the ability to switch between models. Bonus points if you can demonstrate use of a tool!

If you feel bold, see if you can add audio input so you can talk to it, and have it respond with audio. ChatGPT or Claude can help you, or email me if you have questions.

I will publish a full solution here soon - unless someone beats me to it...

There are so many commercial applications for this, from a language tutor, to a company onboarding solution, to a companion AI to a course (like this one!) I can't wait to see your results.

In [ ]:
import os
from dotenv import load_dotenv

import gradio as gr
from IPython.display import Markdown, display, update_display
from openai import OpenAI
import base64
from io import BytesIO
from PIL import Image
from IPython.display import Audio, display


In [ ]:
MODEL_GPT = 'gpt-4o-mini'
MODEL_LLAMA = 'llama3.2'

In [ ]:
# set up environment
load_dotenv(override=True)
api_key = os.getenv("OPENAI_API_KEY")

# set up clients
openai = OpenAI()
ollama = OpenAI(base_url="http://localhost:11434/v1" , api_key="ollama")

# set up system prompt
system_prompt = "You are a coding tutor. If the user asks you a question, answer it to the point. If you are asked to create a code snippet, generate the code in Python and then explain it shortly."

In [ ]:

def speak(message):
    response = openai.audio.speech.create(
        model="tts-1",
        voice="alloy",
        input=message)

    audio_stream = BytesIO(response.content)
    output_filename = "output_audio.mp3"
    with open(output_filename, "wb") as f:
        f.write(audio_stream.read())

    # Play the generated audio
    display(Audio(output_filename, autoplay=True))

In [ ]:
def chat(message, history, model, audio=False):
    messages = [{"role": "system", "content": system_prompt}] + history + [{"role": "user", "content": message}]
    
    if model == "gpt-4o-mini":
        client = openai
    elif model == "llama3.2":
        client = ollama

    stream = client.chat.completions.create(
        model = model,
        messages = messages,
        stream = True
    )

    response = ""
    for chunk in stream:
        response += chunk.choices[0].delta.content or ""
        yield response

    if audio:
        speak(response)


In [ ]:
with gr.Blocks() as ui:
    model_selection = gr.Dropdown(["gpt-4o-mini", "llama3.2"], label="Select model", value="llama3.2")
    audio_enabled = gr.Checkbox(label="Audio enabled", info="Check the box if you want me to speak to you")
    
    gr.ChatInterface(
        title="Coding Friend",
        fn=chat,
        additional_inputs=[model_selection, audio_enabled],
        type="messages"
        )
ui.launch(inbrowser=True)